<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-13 12:41:28
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.27 C
-------------------
Today PnL: 42.98 K (0.34%)
Current PnL: -18.85 L (-13.36%)
CY Booked + Current PnL: -15.98 L (-11.33%)
-------------------
Total profit:  3.80 L
Total loss:  -22.64 L
-------------------
Total Booked + Current PnL: 11.00 L (9.65%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.26%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 80.72 L (63.65%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.42 C (112.22%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.7%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,14.22,X-MC,89.36,140165.0,25387.0,10344.0,0.17,22.12,7.38,31.13,215.0,2.45,1.14,36.53,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-1.02,H-LC,82.98,147126.0,21421.0,14580.0,0.00,17.04,9.91,28.63,116.0,1.47,1.20,31.07,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-16.45,X-LC,47.87,92574.0,17452.0,14710.0,0.30,23.23,15.89,42.81,87.0,1.19,0.75,16.10,X40,BTT,CHEMICALS
70,SBIN,760.30,863.00,-13.45,M-LC,57.45,212450.0,10970.0,16252.0,0.01,5.44,7.65,13.51,91.0,0.67,1.73,16.39,XY25,NTT,BANKS
25,DMART,3484.00,5112.00,-20.59,X-LC,13.83,97805.0,14189.0,24882.0,1.29,16.97,25.44,46.73,123.0,0.57,0.80,19.72,AR,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TANLA,917.30,1963.11,-42.22,H-SC,86.17,141671.0,-97744.0,370696.0,9.78,-40.83,261.66,114.01,29.0,-0.26,1.15,29.93,AR,ATH,IT
88,VALIANTORG,512.64,838.00,-305.24,H-SC,6.38,87360.0,-45926.0,130516.0,5.96,-34.46,149.40,63.47,83.0,-0.35,0.71,46.44,XR,NTT,CHEMICALS
76,STARHEALTH,551.56,761.00,-12.42,M-SC,46.81,143258.0,-56958.0,132986.0,5.69,-28.45,92.83,37.97,46.0,-0.43,1.17,15.02,XY24,NTT,INSURANCE
63,REPCOHOME,515.07,880.00,-56.78,H-SC,78.72,160135.0,-44863.0,190112.0,3.38,-21.88,118.72,70.85,130.0,-0.24,1.30,29.27,XY24,NTT,FINANCE
12,BANKINDIA,113.49,190.00,-34.31,M-MC,41.49,129448.0,296.0,86769.0,3.32,0.23,67.03,67.42,71.0,0.00,1.05,25.90,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INFY,1432.81,2275.00,-12.06,X-LC,79.79,219300.0,20139.0,96931.0,-3.02,10.11,44.20,58.78,9.0,0.21,1.79,14.50,X40,BTT,IT
30,HAPPSTMNDS,919.80,1491.84,-26.03,M-SC,51.06,62286.0,-34293.0,94357.0,-2.71,-35.51,151.49,62.19,22.0,-0.36,0.51,9.21,AR,ATH,IT
15,BRITANNIA,4902.95,6446.05,9.78,X-LC,38.83,218940.0,22822.0,38906.0,-2.41,11.64,17.77,31.47,147.0,0.59,1.78,19.63,AR,ATH,FMCG
19,CERA,8421.60,10420.45,-18.62,X-SC,88.30,83941.0,-25540.0,51523.0,-2.38,-23.33,61.38,23.73,122.0,-0.50,0.68,27.05,X40N,ATH,CERAMICS
84,TTKPRESTIG,769.29,1149.24,-11.32,X-SC,4.26,81010.0,-19767.0,69539.0,-2.13,-19.61,85.84,49.39,162.0,-0.28,0.66,5.42,X40N,ATH,DURABLES


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TATAMOTORS,776.02,1065.00,-52.64,M-LC,93.62,189382.0,-16263.0,92835.0,-0.85,-7.91,49.02,37.24,1.0,-0.18,1.54,23.27,XY24,NTT,AUTO
2,ABB,5319.37,8953.12,-38.02,H-LC,75.53,141762.0,8778.0,82066.0,1.49,6.60,57.89,68.31,2.0,0.11,1.15,15.60,AR,ATH,ELECTRICAL
24,DLF,683.35,1031.70,-26.21,H-LC,43.62,122274.0,-729.0,63436.0,-0.18,-0.59,51.88,50.98,3.0,-0.01,1.00,11.72,AR,ATH,REALTY
10,AWL,342.88,485.00,-69.16,H-MC,5.32,167354.0,-50375.0,140628.0,0.23,-23.14,84.03,41.45,5.0,-0.36,1.36,11.79,XY24,NTT,FMCG
92,WHIRLPOOL,1151.42,2270.00,-35.80,M-SC,95.74,161836.0,15606.0,126459.0,0.24,10.67,78.14,97.15,7.0,0.12,1.32,38.26,X40,NTT,DURABLES


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,14.22,X-MC,89.36,140165.0,25387.0,10344.0,0.17,22.12,7.38,31.13,215.0,2.45,1.14,36.53,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,6.44,X-MC,14.89,187598.0,32163.0,38270.0,0.58,20.69,20.40,45.32,198.0,0.84,1.53,27.65,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-16.45,X-LC,47.87,92574.0,17452.0,14710.0,0.30,23.23,15.89,42.81,87.0,1.19,0.75,16.10,X40,BTT,CHEMICALS
57,POLYCAB,5005.92,7541.35,7.93,H-MC,94.68,168924.0,28758.0,42231.0,0.68,20.52,25.00,50.65,54.0,0.68,1.38,28.92,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-2.22,M-LC,87.23,188992.0,37786.0,27876.0,1.28,24.99,14.75,43.43,37.0,1.36,1.54,31.00,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,POLYCAB,5005.92,7541.35,7.93,H-MC,94.68,168924.0,28758.0,42231.0,0.68,20.52,25.00,50.65,54.0,0.68,1.38,28.92,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,-0.62,H-SC,67.02,161688.0,10324.0,35458.0,0.02,6.82,21.93,30.25,131.0,0.29,1.32,27.97,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-2.22,M-LC,87.23,188992.0,37786.0,27876.0,1.28,24.99,14.75,43.43,37.0,1.36,1.54,31.00,XY24,NTT,MISC
93,WIPRO,243.46,420.00,-8.35,M-LC,52.13,156866.0,5921.0,103532.0,-1.66,3.92,66.00,72.51,58.0,0.06,1.28,18.11,XR,NTT,IT
18,CAMS,3643.00,5250.99,-9.33,H-SC,8.51,102043.0,39.0,44981.0,-0.74,0.04,44.08,44.14,47.0,0.00,0.83,18.31,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
93,WIPRO,243.46,420.00,-8.35,M-LC,52.13,156866.0,5921.0,103532.0,-1.66,3.92,66.00,72.51,58.0,0.06,1.28,18.11,XR,NTT,IT
18,CAMS,3643.00,5250.99,-9.33,H-SC,8.51,102043.0,39.0,44981.0,-0.74,0.04,44.08,44.14,47.0,0.00,0.83,18.31,XR,ATH,MISC
37,INDIAMART,2311.97,4911.36,-52.96,H-SC,74.47,101702.0,-25.0,114394.0,0.79,-0.02,112.48,112.43,NaN,-0.00,0.83,20.03,AR,ATH,MISC
91,VOLTAS,1272.00,1929.20,-29.90,H-MC,20.21,140129.0,209.0,72082.0,0.65,0.15,51.44,51.67,19.0,0.00,1.14,5.84,AR,ATH,AC
12,BANKINDIA,113.49,190.00,-34.31,M-MC,41.49,129448.0,296.0,86769.0,3.32,0.23,67.03,67.42,71.0,0.00,1.05,25.90,XR,NTT,BANKS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UJJIVANSFB,52.77,60.00,36.52,M-SC,80.85,118719.0,-23760.0,43285.0,2.16,-16.68,36.46,13.70,219.0,-0.55,0.97,40.12,X40N,NTT,BANKS
53,MRF,114777.60,150504.14,14.22,X-MC,89.36,140165.0,25387.0,10344.0,0.17,22.12,7.38,31.13,215.0,2.45,1.14,36.53,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-1.02,H-LC,82.98,147126.0,21421.0,14580.0,0.00,17.04,9.91,28.63,116.0,1.47,1.20,31.07,AR,ATH,INSURANCE
86,UNIONBANK,123.87,163.00,-15.64,M-LC,64.89,148799.0,7959.0,36530.0,2.15,5.65,24.55,31.59,83.0,0.22,1.21,29.25,XY24,NTT,BANKS
52,MEDANTA,1022.60,1486.00,6.44,X-MC,14.89,187598.0,32163.0,38270.0,0.58,20.69,20.40,45.32,198.0,0.84,1.53,27.65,XY24,NTT,HEALTHCARE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.0,-805.27,H-SC,100.00,67148.0,-28168.0,72862.0,0.12,-29.55,108.51,46.89,51.0,-0.39,0.55,32.44,X40N,NTT,MISC
79,TATAELXSI,7332.28,9000.0,-19.68,X-MC,98.94,84315.0,-18337.0,41685.0,-1.27,-17.86,49.44,22.74,33.0,-0.44,0.69,26.73,X40N,NTT,IT
58,QUESS,711.66,986.0,NaN,NaN,97.87,79716.0,-76138.0,136219.0,2.54,-48.85,170.88,38.55,NaN,-0.56,0.65,22.07,XY24,NTT,MISC
1,AARTIIND,487.04,919.0,36.75,M-SC,96.81,78515.0,-2821.0,74958.0,0.33,-3.47,95.47,88.69,118.0,-0.04,0.64,32.40,XR,NTT,CHEMICALS
92,WHIRLPOOL,1151.42,2270.0,-35.80,M-SC,95.74,161836.0,15606.0,126459.0,0.24,10.67,78.14,97.15,7.0,0.12,1.32,38.26,X40,NTT,DURABLES


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.30
1,25,41.06
2,50,69.58


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.77
X40     14.52
X40N    13.99
AR      13.83
XR      10.36
XY25    10.27
X200     1.21
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.22
X-LC    18.31
M-SC    16.60
M-LC     8.47
H-MC     7.88
X-MC     6.80
H-LC     5.69
M-MC     5.51
X-SC     2.78
L-SC     2.27
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.81,-2.29,34.82
BANKS,11.11,-10.35,64.48
IT,9.30,-21.40,89.13
FINANCE,7.43,-21.29,76.02
MISC,7.19,-28.45,83.72
INSURANCE,6.12,-2.79,33.78
PAINTS,5.58,-20.41,56.04
ELECTRICAL,4.12,-10.02,66.09
REFINERIES,3.66,9.37,7.33


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3083900.0,27
XR,1271932.0,15
AR,1258632.0,15
X40N,1216698.0,17
X40,811415.0,12
SR,209927.0,2
XY25,150486.0,5
X200,69407.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2462558.0,22
M-SC,1969444.0,20
X-LC,736354.0,12
M-MC,590958.0,5
H-MC,428515.0,7
X-SC,366808.0,4
M-LC,340471.0,6
X-MC,320645.0,6
L-SC,318472.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      942458.0      6
M-SC       XY24      779030.0      6
H-SC       X40N      566209.0      7
           AR        546951.0      3
M-SC       XR        433062.0      5
M-MC       XY24      417772.0      3
X-LC       X40       378180.0      5
M-SC       X40N      376650.0      5
H-SC       XR        304240.0      4
M-LC       XY24      220687.0      4
M-SC       X40       199936.0      2
H-LC       AR        192684.0      4
X-LC       AR        189798.0      3
H-MC       XY24      184850.0      2
L-SC       XR        163647.0      2
X-MC       XY24      140054.0      2
X-SC       SR        123518.0      1
           XR        122228.0      1
           X40N      121062.0      2
X-MC       X40N      110546.0      2
M-LC       XR        103532.0      1
H-SC       X40       102700.0      2
X-LC       XY25       98969.0      3
M-SC       AR         94357.0      1
L-SC       XY24       88826.0      1
M-MC       XR         86769.0      1
           AR         86417.0      1
M-SC       SR         86409.0      1
H-MC       AR         72082.0      1
           X40        70898.0      2
X-LC       X200       69407.0      1
L-SC       AR         65999.0      1
X-MC       X40        59701.0      1
H-MC       XR         58454.0      1
           X40N       42231.0      1
H-LC       XY25       35265.0      1
M-LC       XY25       16252.0      1
X-MC       AR         10344.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
